<a href="https://colab.research.google.com/github/guialmussa-phd/ClimateCodes/blob/main/An%C3%A1lise_de_risco_para_a_produ%C3%A7%C3%A3o_cafeeira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================================================
# Análise de Risco Climático em Regiões Cafeeiras
# Exemplo de pipeline (dados fictícios)
# Guilherme A. L. Torres
# ========================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# --------------------------------------------------------
# Dados fictícios: séries diárias de clima + produção anual
np.random.seed(42)
datas = pd.date_range("2000-01-01", "2019-12-31", freq="D")

df = pd.DataFrame({
    "Data": datas,
    "Tmax": np.random.normal(28, 4, len(datas)),  # Temperatura máxima diária
    "Tmin": np.random.normal(17, 3, len(datas)),  # Temperatura mínima diária
    "Precipitacao": np.random.gamma(2, 2, len(datas))  # chuva diária (mm)
})
df.set_index("Data", inplace=True)

# Produção anual fictícia (sacas/ha)
producao = pd.DataFrame({
    "Ano": np.arange(2000, 2020),
    "Producao_cafe": np.random.normal(25, 3, 20)  # sacas/ha
})

# --------------------------------------------------------
# Índices climáticos
# 1. Consecutive Dry Days (CDD)
df["Dry"] = df["Precipitacao"] < 1.0
df["CDD"] = df["Dry"] * (df["Dry"].groupby((~df["Dry"]).cumsum()).cumcount() + 1)
CDD_annual = df["CDD"].resample("Y").max()

# 2. Summer Days (SU) - dias com Tmax > 30°C
SU_annual = (df["Tmax"] > 30).resample("Y").sum()

# 3. Precipitação total anual
Prec_annual = df["Precipitacao"].resample("Y").sum()

# --------------------------------------------------------
# Unindo índices climáticos e produção
risk = pd.DataFrame({
    "Ano": Prec_annual.index.year,
    "CDD": CDD_annual.values,
    "SU": SU_annual.values,
    "Precipitacao": Prec_annual.values
}).merge(producao, on="Ano")

print(risk.head())

# --------------------------------------------------------
# Correlação entre índices climáticos e produção
corr = risk.corr()
print("\nMatriz de correlação:")
print(corr["Producao_cafe"])

# --------------------------------------------------------
# Visualizações
# 1. Séries temporais
plt.figure(figsize=(12,6))
plt.plot(risk["Ano"], risk["Producao_cafe"], marker="o", label="Produção de café (sacas/ha)")
plt.plot(risk["Ano"], risk["CDD"], marker="s", label="CDD (dias secos consecutivos)")
plt.plot(risk["Ano"], risk["SU"], marker="^", label="SU (dias > 30°C)")
plt.legend()
plt.title("Risco Climático e Produção de Café")
plt.ylabel("Índice / Produção")
plt.grid(True)
plt.show()

# 2. Dispersão: Produção vs CDD
plt.figure(figsize=(7,5))
sns.regplot(x="CDD", y="Producao_cafe", data=risk, ci=None, scatter_kws={"s":70})
plt.title("Produção de café vs Dias secos consecutivos (CDD)")
plt.xlabel("CDD (dias)")
plt.ylabel("Produção de café (sacas/ha)")
plt.show()

# 3. Dispersão: Produção vs SU
plt.figure(figsize=(7,5))
sns.regplot(x="SU", y="Producao_cafe", data=risk, ci=None, scatter_kws={"s":70}, color="red")
plt.title("Produção de café vs Dias > 30°C (SU)")
plt.xlabel("SU (dias)")
plt.ylabel("Produção de café (sacas/ha)")
plt.show()
